In [2]:
import datetime


from cogment_lab.actors import RandomActor, ConstantActor
from cogment_lab.envs.pettingzoo import AECEnvironment
from cogment_lab.process_manager import Cogment
from cogment_lab.utils.runners import process_cleanup


In [3]:
# Cleans up potentially hanging background processes from previous runs
process_cleanup()

Processes terminated successfully.


In [4]:
logpath = f"logs/logs-{datetime.datetime.now().isoformat()}"

cog = Cogment(log_dir=logpath)

print(logpath)

logs/logs-2023-12-13T22:10:04.992766


/Users/ariel/PycharmProjects/cogment_lab/venv/lib/python3.10/site-packages/cogment/context.py:213: UserWarning: No logging handler defined (e.g. logging.basicConfig)
  warnings.warn("No logging handler defined (e.g. logging.basicConfig)")


In [5]:
# Launch an environment in a subprocess

cenv = AECEnvironment(env_path="pettingzoo.butterfly.cooperative_pong_v5.env",
                      render=True)

await cog.run_env(env=cenv, 
                  env_name="pong",
                  port=9011, 
                  log_file="env.log")


True

In [6]:
# Launch two dummy actors in subprocesses

random_actor = RandomActor(cenv.env.action_space("paddle_0"))
constant_actor = ConstantActor(1)

await cog.run_actor(actor=random_actor, 
                    actor_name="random", 
                    port=9021, 
                    log_file="actor-random.log")

await cog.run_actor(actor=constant_actor,
                    actor_name="constant",
                    port=9022,
                    log_file="actor-constant.log")


True

In [7]:
# Check what's running

cog.processes

{'pong': <SpawnProcess name='SpawnProcess-1' pid=19367 parent=19318 started>,
 'random': <SpawnProcess name='SpawnProcess-2' pid=19372 parent=19318 started>,
 'constant': <SpawnProcess name='SpawnProcess-3' pid=19374 parent=19318 started>}

In [8]:
PONG_ACTIONS = ["no-op", "ArrowUp", "ArrowDown"]

actions = PONG_ACTIONS
await cog.run_web_ui(actions=actions, log_file="human.log", fps=60)

True

In [12]:
# Get data from a random + random trial
# You can change the values in `actor_impls` between `web_ui`, `random`, and `constant` to see the different behaviors

trial_id = await cog.start_trial(
    env_name="pong",
    session_config={"render": True},
    actor_impls={
        "paddle_0": "constant",
        "paddle_1": "web_ui",
    },
)

data = await cog.get_trial_data(trial_id)


In [17]:
data["paddle_1"].actions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 1, 1, 1, 0, 2, 2,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])